<img src="EMODnet_compact_colour.png" align="right" width="50%"></img>
# EMODnet OGC services Workshop

In [1]:
from owslib.wmts import WebMapTileService
from ipyleaflet import Map, ImageOverlay
from folium.features import image_to_url
from matplotlib.image import imread
import io
import math

## Part 4: WMTS SERVICES

In [2]:
Bathymetry = 'https://tiles.emodnet-bathymetry.eu/wmts/1.0.0/WMTSCapabilities.xml'
wmts = WebMapTileService(Bathymetry, version='1.0.0')

In [3]:
print(wmts.identification.type)
print(wmts.identification.version)
print(wmts.identification.title)
print(wmts.identification.abstract)

OGC WMTS
1.0.0
EMODnet Bathymetry
None


#### Find out what layers are available

In [15]:
list(wmts.contents)

['baselayer',
 'baselayer_land',
 'mean_atlas_land_2016',
 'mean_atlas_land_2018',
 'mean_atlas_land_2020',
 'mean_atlas_land_2022',
 'mean_atlas_land_latest',
 'mean_multicolour_2016',
 'mean_multicolour_2018',
 'mean_multicolour_2020',
 'mean_multicolour_2022',
 'mean_multicolour_latest',
 'mean_rainbowcolour_2016',
 'mean_rainbowcolour_2018',
 'mean_rainbowcolour_2020',
 'mean_rainbowcolour_2022',
 'mean_rainbowcolour_latest']

#### Get the details of a layer (available layer metadata):

In [16]:
dataset = 'baselayer'

In [17]:
print(wmts[dataset].title)
print(wmts[dataset].abstract)

Global land and water coverage
30 arc second compilation of EMODnet Bathymetry 2018, GEBCO 2019 and various land DEM sources (ASTER, SRTM, EU DEM and viewfinderspanoramas.org) in traditional atlas style colours


In [18]:
print([wmts.tilematrixsets[name].identifier for name in wmts.tilematrixsets])

['inspire_quad', 'web_mercator', 'epsg_3031', 'epsg_3996', 'laea']


#### Get bounding boxes information in WGS84
Save as variable 'bbox1 & 2' and print to inspect

In [19]:
#Print the Bounding box of the desired layers
bbox1 = wmts[dataset].boundingBoxWGS84

print(bbox1)


(-180.0, -90.0, 180.0, 90.0)


#### Get available styles

In [20]:
wmts[dataset].styles

{'default': {'isDefault': False,
  'legend': 'https://tiles.emodnet-bathymetry.eu/legends/legend_atlas_land.png',
  'width': '198',
  'height': '275',
  'format': 'image/png'}}

#### See available methods

In [21]:
[op.name for op in wmts.operations]

[]

## Web Map Tiles services Access.
Definition and example usage of the functions needed to plot the WMTS. Here, we plot the simple OSM baselayer that was explored above. 
(please note that WMTS visualisation is not supported on GitHub, to view the interavtive map layer launch the tutorial through Binder).

In [22]:
class BasemapDefinition:
    def __init__(self, lat, lon, zoom=10):
        
        self.lat_center = lat
        self.lon_center = lon
        
        try:
            self.zoom = zoom
        except:
            pass
        
                
        self.map = Map(center=[self.lat_center, self.lon_center],
                       zoom=self.zoom,
                       width='50%',
                       heigth=6000)
    
    def wmtsRequest(self, coordinates_list, layers_list):
        wmts_url = Bathymetry
        wmts = WebMapTileService(wmts_url)
        
        for lat, lon in coordinates_list:
            x, y = self.deg2num(lat, lon, self.zoom)
            for layer in layers_list:
                wmtsOut = wmts.gettile(layer=layer,
                                        tilematrixset='web_mercator',
                                        tilematrix=self.zoom,
                                        row=y,
                                        column=x,
                                        format="image/png")
                imgArr = imread(io.BytesIO(wmtsOut.read()))
                lat_max, lon_min = self.num2deg(x, y, self.zoom)
                lat_min, lon_max = self.num2deg(x+1, y+1, self.zoom)
                imgurl = image_to_url(image=imgArr)
                self.map.add_layer(ImageOverlay(url=imgurl,
                                                bounds=[[lat_min, lon_min],
                                                        [lat_max, lon_max]]))

    def deg2num(self, lat, lon, zoom):
        
        lat_rad = math.radians(lat)
        n = 2.0 ** zoom
        xtile = int((lon + 180.0) / 360.0 * n)
        ytile = int((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi) / 2.0 * n)
        return (xtile, ytile)
    
    def num2deg(self, xtile, ytile, zoom):
        """returns NW corners, for other corners, use xtile+1 and/or
        ytile+1"""
        n = 2.0 ** zoom
        lon_deg = xtile / n * 360.0 - 180.0
        lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
        lat_deg = math.degrees(lat_rad)
        return (lat_deg, lon_deg)

bd = BasemapDefinition(45, 0, 4)
coordinates_list = [(45, 0), (45,-5), (45, 30), (35, 0), (35,-5),(35, 30)]  # Add more coordinates as needed
layers_list = [dataset]  # Add more layers as needed
bd.wmtsRequest(coordinates_list, layers_list)
bd.map

Map(center=[45, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…